In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import random 

from scipy.stats import uniform, randint
from helpers.model_utils import train, predict, evaluate, random_search
from helpers.load_manager import load_dataframe, load_data
from helpers.plots import plot_losses, plot_predictions_vs_labels, plot_predictions_vs_labels_by_species\
, plot_boxplot_predictions_vs_labels, plot_hexbin_predictions_vs_labels
from models.xgb_model import XGBoostModel
from sklearn.preprocessing import OneHotEncoder

# from ray import tune
# from ray.tune.schedulers import ASHAScheduler

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
data_df

,Unnamed: 0,Species,Chromosome,Region,As_1 (GE) - TPM,As_2 (GE) - TPM,As_3 (GE) - TPM,Bs_1 (GE) - TPM,Bs_2 (GE) - TPM,Bs_3 (GE) - TPM,...,Sp_2 (GE) - TPM,Sp_3 (GE) - TPM,Tm_1 (GE) - TPM,Tm_2 (GE) - TPM,Tm_3 (GE) - TPM,Vic_1 (GE) - TPM,Vic_2 (GE) - TPM,Vic_3 (GE) - TPM,Species ID,upstream200
0,0,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(1..400),4579.764521,2874.021275,2805.289041,3462.302060,3501.986142,3502.851956,...,8965.167866,9056.547435,3205.688636,3228.682954,3103.641082,3401.841238,3832.566805,3408.199122,ACHX,CCTTCCAAGCTTACGACGAGGGTTCGATTCCCTTCACCCGCTCCAA...
1,1,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(980..2143),14.339311,17.782257,23.076558,14.063192,17.260141,12.550914,...,12.322023,9.714290,30.958629,32.159798,33.681898,22.375684,22.529990,19.090084,ACHX,GCATTGCATCTCGGCCGCCAGTTGCGCACGGGCACCGTCTTCATGA...
2,2,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(2164..3555),8.624839,10.448941,3.816945,6.082623,6.523412,3.794404,...,1.545564,3.046184,10.307187,11.364142,15.717096,6.735852,5.521991,4.961554,ACHX,GGAAGCGGCGCAGCGCCTGCGCCAGTCCGAAGCCGCGCGCGAGCTG...
3,3,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(3599..4984),21.127260,38.747719,23.426751,15.679651,17.443773,17.756848,...,3.621928,3.824213,46.222022,37.725002,48.744498,29.690886,27.566362,33.364651,ACHX,CGAAGGCGCGGGCGGCGTCATTACCGATTGGTCGGGGCAGCCGTTA...
4,4,Achromobacter xylosoxidans SOLR10,NZ_CP025774,complement(5049..5855),338.906377,344.525203,239.579899,222.429542,237.845060,159.168702,...,8.886514,10.508767,115.769842,95.915567,75.475714,208.814208,333.092076,332.654255,ACHX,AAGGAATACACGGCAACGGCGATGAGCTGCTTTTCTGCGGACGGCG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96406,96406,Streptococcus suis S10 -P1/7,NC_012925,2000927..2003506,154.651786,166.363005,158.597735,103.096554,117.077900,113.354978,...,NaN,NaN,186.116702,195.310415,191.215538,121.440741,149.782243,68.981909,SSUIS,TGGCCATTACGACTGTTGCGAACAAAGTTCGCCTTATCTCCAACCT...
96407,96407,Streptococcus suis S10 -P1/7,NC_012925,complement(2003907..2004590),51.843037,47.737511,50.150766,30.035985,27.269205,22.901773,...,NaN,NaN,69.873527,55.227042,67.758348,94.401511,108.957265,44.032987,SSUIS,ATATAACTTTTATCATCGGTGGTAGTTTGGGTCTCGATTCTTGTAT...
96408,96408,Streptococcus suis S10 -P1/7,NC_012925,complement(2004615..2005094),72.526107,104.306461,90.287062,67.168967,49.922529,76.483851,...,NaN,NaN,161.530080,151.730774,194.563257,245.576984,273.601934,308.030757,SSUIS,TTTTTTCATATAAGGAATCCTTTCCATTTCTTTAATTTTCCTTAAG...
96409,96409,Streptococcus suis S10 -P1/7,NC_012925,2005268..2006464,133.774758,140.802925,145.617949,191.050633,234.493519,234.235125,...,NaN,NaN,180.352408,159.162080,162.029477,143.215049,132.640597,134.195769,SSUIS,TCCAACGGTAATCAATTTTATTTTCATAAAATAATTGTAACATATC...


In [27]:
data_df = load_dataframe()
data_df

In [39]:
data_df.reset_index(drop=True, inplace= True)

In [40]:
data_df

,Species ID,upstream200,Stress,Stress_name
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,...",8.757449,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,...",8.064743,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,...",8.152174,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,...",8.157670,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,...",8.174290,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
...,...,...,...,...
1078329,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,...",5.805207,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
1078330,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,...",5.170262,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
1078331,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,...",5.666854,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
1078332,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,...",5.717680,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [41]:
species_flat = flatten_columns(data_df, 'Species ID')
stress_name_flat = flatten_columns(data_df, 'Stress_name')
upstream_flat = pd.DataFrame(np.vstack(data_df['upstream200'].values), index=data_df.index)
stress_flat = pd.DataFrame(data_df['Stress_name'].values,columns=['Stress'], index=data_df.index)

ValueError: Shape of passed values is (218901802, 4), indices imply (1078334, 4)

In [34]:
def flatten_columns(df, col_name):
    """Flatten columns containing lists into separate columns"""
    # Create a DataFrame from the lists in the column
    col_flattened = pd.DataFrame(df[col_name].tolist(), index=df.index)
    # Rename the columns to indicate the original column and position
    col_flattened.columns = [f'{col_name}_{i}' for i in range(col_flattened.shape[1])]
    return col_flattened

# Apply the helper function to each relevant column
species_flat = flatten_columns(data_df, 'Species ID')
stress_name_flat = flatten_columns(data_df, 'Stress_name')
upstream_flat = pd.DataFrame(np.vstack(data_df['upstream200'].values), index=data_df.index)
stress_flat = pd.DataFrame(data_df['Stress_name'].values,columns=['Stress'], index=data_df.index)

# Concatenate the flattened columns
data_flattened = pd.concat([species_flat, stress_name_flat, upstream_flat, stress_flat], axis=1)

print(data_flattened.dtypes)  # Verify all columns are now of type int
print(data_flattened.head())  # Check the first few rows of the flattened DataFrame


ValueError: Shape of passed values is (218901802, 4), indices imply (1078334, 4)

In [ ]:
def prepare_data(data_flattened, labels):
    # Combine the features and labels into a single DataFrame
    data_flattened['Stress'] = labels
    
    y = data_flattened.pop('Stress').values
    X = data_flattened.values

    # Split the data into training, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(data_flattened, train_dataset.y)

In [4]:
config = {
    'lr': 0.001,
    'batch_size': 1024,
    'epochs': 10,
    'species_id': -1,
    'test_size': 20000,
    'hidden_size': 60,
    'cnn_filters': 100,
    'model_version': 1,
    'data_df': data_df
}

In [12]:
train_dataset.X

,Species ID,Stress_name,upstream200
71049,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
31521,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
9035,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [1,..."
23799,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0], [1,..."
23036,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[[1, 0, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,..."
...,...,...,...
93908,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,..."
9189,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[1, 0, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,..."
22498,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,..."
30864,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,..."


In [ ]:
model = XGBoostModel(max_depth=6, lr=0.1, n_estimators=100)
model.fit(X_train, y_train)

In [21]:
train_dataset.X

,Species ID,Stress_name,upstream200
71049,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
31521,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
9035,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [1,..."
23799,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 1, 0, 0], [1,..."
23036,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[[1, 0, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,..."
...,...,...,...
93908,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,..."
9189,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[1, 0, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,..."
22498,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,..."
30864,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,..."
